In [17]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, matthews_corrcoef,
    roc_curve, precision_recall_curve, auc
)

# ====== CONFIG ======
ratios_dir = "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_fixed_rfe"
baseline_dirs = [
    "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_BASELINE_demo_only",
    "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_BASELINE_random",
    "/Users/adithyamadduri/Desktop/Projects/proteomics_LGBM(ANML+Meta)_fixed"
]

model_names = [
    "Proteomic Ratios",
    "Demographics Only Baseline",
    "Stratified Random Baseline",
    "Raw Proteins + Demographics"
]

classes = ["MCI", "NCI", "AD", "AD+"]
seeds = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace("+", "plus").replace(" ", "_").replace("/", "-")


In [18]:
def compute_metrics(y_true, y_score, y_pred):
    """Compute all metrics for a given set of predictions."""
    metrics = {}
    
    # Basic classification metrics
    metrics["Accuracy"] = accuracy_score(y_true, y_pred)
    metrics["Precision"] = precision_score(y_true, y_pred, zero_division=0)
    metrics["Recall"] = recall_score(y_true, y_pred, zero_division=0)
    metrics["F1_Score"] = f1_score(y_true, y_pred, zero_division=0)
    metrics["MCC"] = matthews_corrcoef(y_true, y_pred)
    
    # ROC-AUC (using exact method from AD_Ratios_Plotting)
    if len(np.unique(y_true)) > 1 and y_true.sum() > 0 and y_true.sum() < len(y_true):
        metrics["AUC"] = roc_auc_score(y_true, y_score)
    else:
        metrics["AUC"] = np.nan
    
    # Average Precision (using exact method from AD_Ratios_Plotting)
    if len(np.unique(y_true)) > 1 and y_true.sum() > 0:
        metrics["AP"] = average_precision_score(y_true, y_score)
    else:
        metrics["AP"] = np.nan
    
    return metrics


In [19]:
def calculate_model_metrics(results_dir, classes, seeds):
    """Calculate mean metrics across seeds for each class."""
    all_results = []
    
    for cls in classes:
        seed_metrics = []
        
        for seed in seeds:
            file_path = os.path.join(results_dir, f"seed{seed}_{safe_cls(cls)}.csv")
            
            if not os.path.exists(file_path):
                print(f"Missing: {file_path}")
                continue
            
            df = pd.read_csv(file_path)
            
            if len(df) == 0:
                continue
            
            y_true = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values
            
            # Get binary predictions using 0.5 threshold
            y_pred = (y_score >= 0.5).astype(int)
            
            # Compute metrics
            metrics = compute_metrics(y_true, y_score, y_pred)
            seed_metrics.append(metrics)
        
        # Average across seeds
        if seed_metrics:
            avg_metrics = {}
            for key in seed_metrics[0].keys():
                values = [m[key] for m in seed_metrics if not np.isnan(m[key])]
                if values:
                    avg_metrics[key] = np.mean(values)
                    avg_metrics[f"{key}_std"] = np.std(values)
                else:
                    avg_metrics[key] = np.nan
                    avg_metrics[f"{key}_std"] = np.nan
            
            avg_metrics["Class"] = cls
            all_results.append(avg_metrics)
        else:
            # No valid seeds for this class
            all_results.append({
                "Class": cls,
                "Accuracy": np.nan, "Accuracy_std": np.nan,
                "Precision": np.nan, "Precision_std": np.nan,
                "Recall": np.nan, "Recall_std": np.nan,
                "F1_Score": np.nan, "F1_Score_std": np.nan,
                "MCC": np.nan, "MCC_std": np.nan,
                "AUC": np.nan, "AUC_std": np.nan,
                "AP": np.nan, "AP_std": np.nan
            })
    
    return pd.DataFrame(all_results)


In [20]:
# Calculate metrics for all models
all_model_results = []

# Ratios model
print("Calculating metrics for Proteomic Ratios...")
ratios_df = calculate_model_metrics(ratios_dir, classes, seeds)
ratios_df["Model"] = model_names[0]
all_model_results.append(ratios_df)

# Baseline models
for i, baseline_dir in enumerate(baseline_dirs):
    print(f"Calculating metrics for {model_names[i+1]}...")
    baseline_df = calculate_model_metrics(baseline_dir, classes, seeds)
    baseline_df["Model"] = model_names[i+1]
    all_model_results.append(baseline_df)

# Combine all results
results_df = pd.concat(all_model_results, ignore_index=True)

print("\nMetrics calculation complete!")


Calculating metrics for Proteomic Ratios...
Calculating metrics for Demographics Only Baseline...
Calculating metrics for Stratified Random Baseline...
Calculating metrics for Raw Proteins + Demographics...

Metrics calculation complete!


In [21]:
# Reorder columns for better presentation
column_order = ["Model", "Class", "Accuracy", "Accuracy_std",
                "Precision", "Precision_std", "Recall", "Recall_std",
                "F1_Score", "F1_Score_std", "MCC", "MCC_std",
                "AUC", "AUC_std", "AP", "AP_std"]

# Ensure all columns exist
for col in column_order:
    if col not in results_df.columns:
        results_df[col] = np.nan

results_df = results_df[column_order]

# Format the table for display
display_df = results_df.copy()

# Format numeric columns to 4 decimal places
numeric_cols = [col for col in display_df.columns if col not in ["Model", "Class"]]
for col in numeric_cols:
    display_df[col] = display_df[col].apply(lambda x: f"{x:.4f}" if not pd.isna(x) else "N/A")

print("=" * 120)
print("PERFORMANCE METRICS TABLE")
print("=" * 120)
print(f"\nTotal rows: {len(display_df)} (4 models × 4 classes)")
print(f"Metrics averaged across {len(seeds)} seeds: {seeds}")
print("\n" + "=" * 120)
print(display_df.to_string(index=False))
print("\n" + "=" * 120)


PERFORMANCE METRICS TABLE

Total rows: 16 (4 models × 4 classes)
Metrics averaged across 5 seeds: [1, 2, 3, 4, 5]

                      Model Class Accuracy Accuracy_std Precision Precision_std Recall Recall_std F1_Score F1_Score_std     MCC MCC_std    AUC AUC_std     AP AP_std
           Proteomic Ratios   MCI   0.7094       0.0118    0.4638        0.0399 0.2951     0.0544   0.3590       0.0493  0.1925  0.0471 0.6495  0.0293 0.4180 0.0304
           Proteomic Ratios   NCI   0.6667       0.0251    0.6819        0.0320 0.6940     0.0213   0.6875       0.0211  0.3313  0.0510 0.7277  0.0330 0.7268 0.0490
           Proteomic Ratios    AD   0.8335       0.0062    0.5556        0.0396 0.3207     0.0488   0.4024       0.0335  0.3324  0.0198 0.7899  0.0152 0.4998 0.0134
           Proteomic Ratios   AD+   0.9802       0.0061    0.1333        0.1944 0.0900     0.1114   0.1030       0.1350  0.1027  0.1382 0.7684  0.1267 0.3351 0.1650
 Demographics Only Baseline   MCI   0.7200       0.0060    0

In [22]:
# Create a nicely formatted table with mean ± std format
formatted_results = []

for _, row in results_df.iterrows():
    formatted_row = {
        "Model": row["Model"],
        "Class": row["Class"],
        "Accuracy": f"{row['Accuracy']:.4f} ± {row['Accuracy_std']:.4f}" if not pd.isna(row['Accuracy']) else "N/A",
        "Precision": f"{row['Precision']:.4f} ± {row['Precision_std']:.4f}" if not pd.isna(row['Precision']) else "N/A",
        "Recall": f"{row['Recall']:.4f} ± {row['Recall_std']:.4f}" if not pd.isna(row['Recall']) else "N/A",
        "F1_Score": f"{row['F1_Score']:.4f} ± {row['F1_Score_std']:.4f}" if not pd.isna(row['F1_Score']) else "N/A",
        "MCC": f"{row['MCC']:.4f} ± {row['MCC_std']:.4f}" if not pd.isna(row['MCC']) else "N/A",
        "AUC": f"{row['AUC']:.4f} ± {row['AUC_std']:.4f}" if not pd.isna(row['AUC']) else "N/A",
        "AP": f"{row['AP']:.4f} ± {row['AP_std']:.4f}" if not pd.isna(row['AP']) else "N/A"
    }
    formatted_results.append(formatted_row)

formatted_df = pd.DataFrame(formatted_results)

print("\n" + "=" * 120)
print("FORMATTED PERFORMANCE TABLE (Mean ± Std)")
print("=" * 120)
print(formatted_df.to_string(index=False))
print("\n" + "=" * 120)



FORMATTED PERFORMANCE TABLE (Mean ± Std)
                      Model Class        Accuracy       Precision          Recall        F1_Score              MCC             AUC              AP
           Proteomic Ratios   MCI 0.7094 ± 0.0118 0.4638 ± 0.0399 0.2951 ± 0.0544 0.3590 ± 0.0493  0.1925 ± 0.0471 0.6495 ± 0.0293 0.4180 ± 0.0304
           Proteomic Ratios   NCI 0.6667 ± 0.0251 0.6819 ± 0.0320 0.6940 ± 0.0213 0.6875 ± 0.0211  0.3313 ± 0.0510 0.7277 ± 0.0330 0.7268 ± 0.0490
           Proteomic Ratios    AD 0.8335 ± 0.0062 0.5556 ± 0.0396 0.3207 ± 0.0488 0.4024 ± 0.0335  0.3324 ± 0.0198 0.7899 ± 0.0152 0.4998 ± 0.0134
           Proteomic Ratios   AD+ 0.9802 ± 0.0061 0.1333 ± 0.1944 0.0900 ± 0.1114 0.1030 ± 0.1350  0.1027 ± 0.1382 0.7684 ± 0.1267 0.3351 ± 0.1650
 Demographics Only Baseline   MCI 0.7200 ± 0.0060 0.2870 ± 0.3943 0.0278 ± 0.0496 0.0442 ± 0.0765  0.0397 ± 0.0487 0.5508 ± 0.0063 0.3349 ± 0.0129
 Demographics Only Baseline   NCI 0.5833 ± 0.0364 0.5850 ± 0.0414 0.8012 ± 0

In [23]:
# Save to CSV
output_dir = "/Users/adithyamadduri/Desktop/Projects/ratios_project/Finalized_Code/Results"
os.makedirs(output_dir, exist_ok=True)

csv_path = os.path.join(output_dir, "performance_metrics_table.csv")
results_df.to_csv(csv_path, index=False)

formatted_csv_path = os.path.join(output_dir, "performance_metrics_table_formatted.csv")
formatted_df.to_csv(formatted_csv_path, index=False)

print(f"\nSaved detailed metrics to: {csv_path}")
print(f"Saved formatted table to: {formatted_csv_path}")



Saved detailed metrics to: /Users/adithyamadduri/Desktop/Projects/ratios_project/Finalized_Code/Results/performance_metrics_table.csv
Saved formatted table to: /Users/adithyamadduri/Desktop/Projects/ratios_project/Finalized_Code/Results/performance_metrics_table_formatted.csv


In [24]:
# Create a summary table showing best model per class for key metrics
summary_data = []

for cls in classes:
    cls_data = results_df[results_df["Class"] == cls].copy()
    
    # Find best model for each metric
    best_auc_idx = cls_data["AUC"].idxmax() if not cls_data["AUC"].isna().all() else None
    best_ap_idx = cls_data["AP"].idxmax() if not cls_data["AP"].isna().all() else None
    best_f1_idx = cls_data["F1_Score"].idxmax() if not cls_data["F1_Score"].isna().all() else None
    
    summary_data.append({
        "Class": cls,
        "Best_AUC_Model": cls_data.loc[best_auc_idx, "Model"] if best_auc_idx is not None else "N/A",
        "Best_AUC_Value": f"{cls_data.loc[best_auc_idx, 'AUC']:.4f}" if best_auc_idx is not None else "N/A",
        "Best_AP_Model": cls_data.loc[best_ap_idx, "Model"] if best_ap_idx is not None else "N/A",
        "Best_AP_Value": f"{cls_data.loc[best_ap_idx, 'AP']:.4f}" if best_ap_idx is not None else "N/A",
        "Best_F1_Model": cls_data.loc[best_f1_idx, "Model"] if best_f1_idx is not None else "N/A",
        "Best_F1_Value": f"{cls_data.loc[best_f1_idx, 'F1_Score']:.4f}" if best_f1_idx is not None else "N/A"
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "=" * 120)
print("SUMMARY: Best Model per Class")
print("=" * 120)
print(summary_df.to_string(index=False))
print("\n" + "=" * 120)

# Save summary
summary_csv_path = os.path.join(output_dir, "performance_summary_best_models.csv")
summary_df.to_csv(summary_csv_path, index=False)
print(f"\nSaved summary to: {summary_csv_path}")



SUMMARY: Best Model per Class
Class   Best_AUC_Model Best_AUC_Value    Best_AP_Model Best_AP_Value    Best_F1_Model Best_F1_Value
  MCI Proteomic Ratios         0.6495 Proteomic Ratios        0.4180 Proteomic Ratios        0.3590
  NCI Proteomic Ratios         0.7277 Proteomic Ratios        0.7268 Proteomic Ratios        0.6875
   AD Proteomic Ratios         0.7899 Proteomic Ratios        0.4998 Proteomic Ratios        0.4024
  AD+ Proteomic Ratios         0.7684 Proteomic Ratios        0.3351 Proteomic Ratios        0.1030


Saved summary to: /Users/adithyamadduri/Desktop/Projects/ratios_project/Finalized_Code/Results/performance_summary_best_models.csv
